### RAG Application
1. Set up a vector store
2. Embed and add documents
3. Query documents
4. Create prompt for LLM
5. Generate response from LLM

# Rag from a PDF

In [ ]:
!pip install langchain

In [ ]:
!pip install -U langchain-cohere

['Collecting langchain-cohere',
 '  Downloading langchain_cohere-0.3.1-py3-none-any.whl.metadata (6.7 kB)',
 'Collecting cohere<6.0,>=5.5.6 (from langchain-cohere)',
 '  Downloading cohere-5.11.4-py3-none-any.whl.metadata (3.5 kB)',
 'Requirement already satisfied: langchain-core<0.4,>=0.3.0 in /usr/local/lib/python3.10/dist-packages (from langchain-cohere) (0.3.17)',
 'Collecting langchain-experimental>=0.3.0 (from langchain-cohere)',
 '  Downloading langchain_experimental-0.3.3-py3-none-any.whl.metadata (1.7 kB)',
 'Requirement already satisfied: pandas>=1.4.3 in /usr/local/lib/python3.10/dist-packages (from langchain-cohere) (2.2.2)',
 'Requirement already satisfied: pydantic<3,>=2 in /usr/local/lib/python3.10/dist-packages (from langchain-cohere) (2.9.2)',
 'Requirement already satisfied: tabulate<0.10.0,>=0.9.0 in /usr/local/lib/python3.10/dist-packages (from langchain-cohere) (0.9.0)',
 'Collecting fastavro<2.0.0,>=1.9.4 (from cohere<6.0,>=5.5.6->langchain-cohere)',
 '  Downloadi

In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 76.0 MB/s eta 0:00:00


In [ ]:
from langchain_cohere import ChatCohere

from langchain_core.messages import HumanMessage, SystemMessage

In [ ]:
import os
import configparser

In [ ]:
config = configparser.ConfigParser()
config.read('/content/config.ini')

cohere = config['cohere']

os.environ['COHERE_API_KEY'] = cohere.get('COHERE_API_KEY')

In [ ]:
import fitz
with fitz.open("attention_all_you_need.pdf") as doc:
    text = ""
    for page in doc:
        text += page.get_text()

print(text)

Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring signiﬁcantly
less time to train. Our model a

In [ ]:
print(text.count("\n"))

1050


In [ ]:
text = text.replace("\n"," ")

In [ ]:
sentence_list = text.split(".")

In [ ]:
len(sentence_list)

567

In [ ]:
import configparser
import os
# set up config parser
config = configparser.ConfigParser()
config.read("/content/config_hf.ini")  # holds secrets and keys


# load Groq config
hf = config["huggingface"]

In [ ]:
model_id = "sentence-transformers/all-MiniLM-L6-v2"

In [ ]:
import requests

api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf.get('hf_token')}"}

In [ ]:

def query(texts):
    response = requests.post(api_url, headers=headers,
                             json={"inputs": texts, "options":{"wait_for_model":True}})
    return response.json()

In [ ]:
sentence_vectorDB = query(sentence_list)

In [ ]:
len(sentence_vectorDB), len(sentence_vectorDB[0])

(567, 384)

In [ ]:
import torch
import numpy as np
sent_embeddings = torch.from_numpy(np.array(sentence_vectorDB)).to(torch.float)

In [ ]:
sent_embeddings.shape

torch.Size([567, 384])

In [ ]:
question = ["what is encoder in transformer ?"]
output = query(question)
query_embeddings = torch.FloatTensor(output)
from sentence_transformers.util import semantic_search

hits = semantic_search(query_embeddings, sent_embeddings, top_k=5)
selectedStory=[]
for i in range(len(hits[0])):
  print(sentence_list[hits[0][i]['corpus_id']])
  selectedStory.append(sentence_list[hits[0][i]['corpus_id']])

 The Transformer follows this overall architecture using stacked self-attention and point-wise, fully connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1, respectively
 To the best of our knowledge, however, the Transformer is the ﬁrst transduction model relying entirely on self-attention to compute representations of its input and output without using sequence- aligned RNNs or convolution
3 Applications of Attention in our Model The Transformer uses multi-head attention in three different ways: • In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder
 Here, the encoder maps an input sequence of symbol representations (x1, 
 In the following sections, we will describe the Transformer, motivate self-attention and discuss its advantages over models such as [17, 18] and [9]


In [ ]:
str(selectedStory)

'[\' The Transformer follows this overall architecture using stacked self-attention and point-wise, fully connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1, respectively\', \' To the best of our knowledge, however, the Transformer is the ﬁrst transduction model relying entirely on self-attention to compute representations of its input and output without using sequence- aligned RNNs or convolution\', \'3 Applications of Attention in our Model The Transformer uses multi-head attention in three different ways: • In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder\', \' Here, the encoder maps an input sequence of symbol representations (x1, \', \' In the following sections, we will describe the Transformer, motivate self-attention and discuss its advantages over models such as [17, 18] and [9]\']'

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

sysMessage = SystemMessage('''Role: You are question answer bot. You need to refer to the story as a list below and answer the users question
                           in an detailed manner.''')
sysMessage1 = SystemMessage(f'Store: {selectedStory}')


humanMessage = HumanMessage(question[0])
# max_tokens=256, temperature=0.75
# https://python.langchain.com/api_reference/cohere/chat_models/langchain_cohere.chat_models.ChatCohere.html#langchain_cohere.chat_models.ChatCohere
model = ChatCohere(model="command-r-plus")
response = model.invoke([sysMessage, sysMessage1, humanMessage])
print(response)

content='The encoder is a crucial component of the Transformer, a powerful model for sequence transduction tasks. In the provided context, the encoder is described as follows: \n\n- The encoder is responsible for mapping an input sequence of symbol representations, denoted as (x1, ...) to a sequence of continuous representations. \n- It uses stacked self-attention and point-wise, fully connected layers to process the input. \n- The output of the encoder serves as the "memory" for the subsequent "encoder-decoder attention" layers. \n- In the "encoder-decoder attention" mechanism, the queries come from the previous decoder layer, while the keys and values are provided by the output of the encoder. \n\nOverall, the encoder plays a vital role in transforming the input data into a format that the Transformer model can use for further processing and generating outputs. It captures the underlying patterns and relationships in the input sequence, enabling the model to perform tasks like machin